In [1]:
import multiprocessing as mp
import papermill as pm

from pathlib import Path
from itertools import product

In [2]:
base_coef_analysis_filepath = {"correlation": "conditional_correlations/analyzed_corr_coef_results_{}.npz",
                              "fitness":"fitness_coefs/analyzed_fitness_coef_results_{}.npz"}
results_dir = '../..'

In [3]:
shorthands = []
output_dirs = []

for experiment_type, control_type in product(["gluttonous", "picky"], repeat=2):
    shorthand = '{}{}'.format(experiment_type[0], control_type[0])
    shorthands.append(shorthand)
    output_dir = 'avg_types_{}'.format(shorthand)
    Path("./" + output_dir).mkdir(parents=True, exist_ok=True)
    output_dirs.append(output_dir)

In [4]:
input_names = ['../notebook_generation_template.ipynb' for shorthand in shorthands]
output_names = ['{}/generate_plots.ag.ipynb'.format(output_dir) for output_dir in output_dirs]
parameter_dicts = [{'results_dir': results_dir,
                    'base_result_filename': {key:value.format(shorthand) for key, value in base_coef_analysis_filepath.items()},
                    'output_dir': output_dir} 
                   for shorthand, output_dir 
                   in zip(shorthands, output_dirs)]

In [5]:
parallel_input = [[input_name, output_name, parameter_dict] 
                  for input_name, output_name, parameter_dict
                  in zip(input_names, output_names, parameter_dicts)]

In [6]:
pool = mp.Pool(mp.cpu_count())
pool.starmap(pm.execute_notebook, parallel_input)
pool.close()
pool.join()